# RAG

Razvoj AI chatbot-a koji će odgovarati na pitanja vezana za procesiranje prirodnog jezika (eng. *Natural Language Processing (NLP)*) oslanjajući se na prezentacije sa predavanja.

## Podešavanje konekcije ka OpenAI

Potrebno je da naš OpenAI API ključ postavimo kao *environment variable*. Ključ se generiše na sledećem [linku](https://platform.openai.com/api-keys).

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

## Podaci

### Učitavanje

Radimo učitavanje podataka iz PDF-ova i radimo njihovo razbijanje na delove.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

In [ ]:
folder_path = "data/"
loader = DirectoryLoader(folder_path, glob="*pdf", loader_cls=PyPDFLoader)
docs = loader.load()

*PyPDF* je odmah odradio razbijanje PDF-ova na stranice. Ispisaćemo ukupan broj stranica i prikazati prvih 5 učitanih.

In [ ]:
print("Broj učitanih stranica: ", len(docs))

In [ ]:
docs[:5]

### Podela teksta

Radimo podelu teksta na delove koji će biti vektorizovani.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
print("Broj delova teksta: ", len(splits))

In [ ]:
splits[:5]

### Vektorizacija

Koristimo OpenAI embedding-e da vektorizujemo tekst i sačuvamo ga u *ChromaDB* vektorskoj bazi podataka.
Prvo ćemo kao primer pokazati kako izgleda embedding i koja mu je dimenzija, a onda ćemo embedding-ovati sve naše delove.

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings()
embedded_texts = embeddings.embed_documents(["Ovo je prva rečenica. Ovo je druga rečenica."])
print("Ukupan broj embeddinga: ", len(embedded_texts))
print("Dimenzionalnost embedding-a: ", len(embedded_texts[0]))
print("Embedding prve rečenice: \n", embedded_texts[0])

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

## Pretraživač informacija

Inicijalizujemo pretraživač informacija (eng. *retriever*) koji će na osnovu prosleđenog teksta dovlačiti 10 najsličnijih delova teksta. 

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [ ]:
retriever.invoke("Koje godine je nastao Word2Vec?")

## Prompt

Definišemo naš prompt koji će prosleđivati instrukcije i dobavljene podatke velikom jezičkom modelu i na kraju vraćati odgovor.

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
template = """Ti si asistent za učenje koji odgovara na studentska pitanja.
Koristi sledeće delove dobavljenog konteksta da odgovoriš na pitanje.
Tvoj stil treba da bude prijateljski, neformalan i informativan, a odgovori koncizni.
Ako ti je potrebno još informacija da odgovoriš na pitanje, traži ih.
Ako ne znaš odgovor, reci da kontaktiraju profesora ili asistenta.
Uvek navedi dokument gde se može pronaći više informacija.

Pitanje: {question}
Kontekst: {context}

Odgovor:"""

prompt = PromptTemplate.from_template(template)

In [ ]:
prompt

## RAG lanac

Definišemo RAG lanac koji će povezati naš prompt, dobavljene podatke i jezički model kako bi odgovorio na zadato pitanje. 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Testiranje

In [ ]:
rag_chain.invoke("Koje godine je nastao Word2Vec?")

### Streaming

In [ ]:
def stream_answer(question):
    for chunk in rag_chain.stream(question):
        print(chunk, end="", flush=True)

In [ ]:
stream_answer("Na čemu se zasniva LLaMA model?")

In [ ]:
stream_answer("Šta je RLHF?")

## Čišćenje memorije

Na kraju rada, brišemo kolekciju iz vektorske baze podataka.

In [ ]:
vectorstore.delete_collection()